In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [2]:
dataDir='/home/kent/git/aiacademy-learning-notebook/Project/ImageCaption/data/coco'
dataType='train2014'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

print(annFile)
!ls -alh $annFile

/home/kent/git/aiacademy-learning-notebook/Project/ImageCaption/data/coco/annotations/instances_train2014.json
-rw-rw-r-- 1 kent kent 318M Feb 19 09:39 /home/kent/git/aiacademy-learning-notebook/Project/ImageCaption/data/coco/annotations/instances_train2014.json


In [3]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=14.15s)
creating index...
index created!


In [4]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
outdoor kitchen furniture food vehicle indoor accessory person appliance animal electronic sports


In [9]:
coco.dataset['info']

{'contributor': 'COCO Consortium',
 'date_created': '2017/09/01',
 'description': 'COCO 2014 Dataset',
 'url': 'http://cocodataset.org',
 'version': '1.0',
 'year': 2014}

In [15]:
coco.dataset.keys()

dict_keys(['info', 'categories', 'licenses', 'images', 'annotations'])

In [48]:
# initialize COCO api for caption annotations
annFile = '{}/annotations/captions_{}.json'.format(dataDir,dataType)
coco_caps=COCO(annFile)

loading annotations into memory...
Done (t=0.75s)
creating index...
index created!


In [49]:
train_set ={}
for img in coco.dataset['images']:
#     print(img)
#     print(img['file_name'])
    
    imgId = img['id']
    
    annIds = coco_caps.getAnnIds(imgIds=imgId)
    anns = coco_caps.loadAnns(annIds)
    
    anns_list = []
    for aann in anns:
#         print(aann['caption'])
        anns_list.append(aann['caption'])
        
    train_set[img['file_name']] = anns_list

len(train_set)

82783

In [50]:
import json
fw = open("train.json",'w')
json.dump(train_set,fw)

In [51]:
!ls -alh ./train.json

-rw-rw-r-- 1 kent kent 26M Feb 19 11:39 ./train.json


# Extract Feature From VGG16

In [2]:
from slacker import Slacker
import os
slack = Slacker(os.environ['SLACK_TOKEN'])
def slack_log(message):
    try:
        slack.chat.post_message('#log-traininglog', "CoCoDatasetPrepare: {}".format(message))
    except:
        print("slack connection error")

slack_log('Initial Logger')   

In [3]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

import h5py
import numpy as np

import json

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load VGG16 Model

In [4]:
# load the model
model = VGG16()
# re-structure the model
model.layers.pop()
model = Model(input=model.inputs, output=model.layers[-1].output)
# summarize
print(model.summary())
# extract features from each photo

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("fc..., inputs=[<tf.Tenso...)`
  """


# Generate VGG16 Feature

In [ ]:
directory = './data/coco/images/train2014/'
image_list = listdir(directory)

f = h5py.File("vgg16_feature.hdf5", "w") 
dset = f.create_dataset("image_feautre", (len(image_list),4096), dtype='f')

fjson = open('image_list.json','w')
json.dump(image_list,fjson)
fjson.close()


images = []

for index, name in enumerate(image_list):
     
    
    # load an image from file
    filename = directory + '/' + name
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    
    images.append(image)
    print('>%d,%s' %(index,name))

    
    if (index+1)%10 ==0 and index > 0 :
        images_array = preprocess_input(np.array(images).reshape((10, 224, 224, 3)))
        
        # get features
        feature = model.predict(images_array, verbose=0)

        # store feature
        dset[index-len(images)+1:index+1] = feature
        images = []


        slack_log('>%d,%s' %(index,name))

images_array = preprocess_input(np.array(images).reshape((len(images), 224, 224, 3)))
feature = model.predict(images_array, verbose=0)    
dset[index-len(images)+1:index+1] = feature
slack_log('>%d,%s' %(index,name))

f.close()

>0,COCO_train2014_000000057828.jpg
>1,COCO_train2014_000000577653.jpg
>2,COCO_train2014_000000519135.jpg
>3,COCO_train2014_000000245528.jpg
>4,COCO_train2014_000000479469.jpg
>5,COCO_train2014_000000387717.jpg
>6,COCO_train2014_000000574953.jpg
>7,COCO_train2014_000000029075.jpg
>8,COCO_train2014_000000463684.jpg
>9,COCO_train2014_000000245272.jpg
>10,COCO_train2014_000000223988.jpg
>11,COCO_train2014_000000135806.jpg
>12,COCO_train2014_000000541004.jpg
>13,COCO_train2014_000000182516.jpg
>14,COCO_train2014_000000249076.jpg
>15,COCO_train2014_000000308701.jpg
>16,COCO_train2014_000000001307.jpg
>17,COCO_train2014_000000558860.jpg
>18,COCO_train2014_000000544018.jpg
>19,COCO_train2014_000000249839.jpg
>20,COCO_train2014_000000158701.jpg
>21,COCO_train2014_000000277914.jpg
>22,COCO_train2014_000000201655.jpg
>23,COCO_train2014_000000179454.jpg
>24,COCO_train2014_000000079498.jpg
>25,COCO_train2014_000000467318.jpg
>26,COCO_train2014_000000273736.jpg
>27,COCO_train2014_000000300057.jpg
>2

>230,COCO_train2014_000000392571.jpg
>231,COCO_train2014_000000435045.jpg
>232,COCO_train2014_000000201596.jpg
>233,COCO_train2014_000000469373.jpg
>234,COCO_train2014_000000413021.jpg
>235,COCO_train2014_000000151609.jpg
>236,COCO_train2014_000000391657.jpg
>237,COCO_train2014_000000224278.jpg
>238,COCO_train2014_000000354376.jpg
>239,COCO_train2014_000000299309.jpg
>240,COCO_train2014_000000197636.jpg
>241,COCO_train2014_000000276479.jpg
>242,COCO_train2014_000000510063.jpg
>243,COCO_train2014_000000539335.jpg
>244,COCO_train2014_000000424370.jpg
>245,COCO_train2014_000000108722.jpg
>246,COCO_train2014_000000420608.jpg
>247,COCO_train2014_000000101330.jpg
>248,COCO_train2014_000000033743.jpg
>249,COCO_train2014_000000498160.jpg
>250,COCO_train2014_000000187846.jpg
>251,COCO_train2014_000000000077.jpg
>252,COCO_train2014_000000271789.jpg
>253,COCO_train2014_000000403750.jpg
>254,COCO_train2014_000000041832.jpg
>255,COCO_train2014_000000240062.jpg
>256,COCO_train2014_000000230735.jpg
>

>460,COCO_train2014_000000192862.jpg
>461,COCO_train2014_000000346196.jpg
>462,COCO_train2014_000000056132.jpg
>463,COCO_train2014_000000490155.jpg
>464,COCO_train2014_000000447083.jpg
>465,COCO_train2014_000000433546.jpg
>466,COCO_train2014_000000284144.jpg
>467,COCO_train2014_000000367724.jpg
>468,COCO_train2014_000000069700.jpg
>469,COCO_train2014_000000150421.jpg
>470,COCO_train2014_000000117821.jpg
>471,COCO_train2014_000000098560.jpg
>472,COCO_train2014_000000099580.jpg
>473,COCO_train2014_000000484467.jpg
>474,COCO_train2014_000000516880.jpg
>475,COCO_train2014_000000244096.jpg
>476,COCO_train2014_000000526432.jpg
>477,COCO_train2014_000000126709.jpg
>478,COCO_train2014_000000327498.jpg
>479,COCO_train2014_000000237954.jpg
>480,COCO_train2014_000000447424.jpg
>481,COCO_train2014_000000401060.jpg
>482,COCO_train2014_000000461939.jpg
>483,COCO_train2014_000000304950.jpg
>484,COCO_train2014_000000022482.jpg
>485,COCO_train2014_000000419349.jpg
>486,COCO_train2014_000000529804.jpg
>

>690,COCO_train2014_000000340727.jpg
>691,COCO_train2014_000000369610.jpg
>692,COCO_train2014_000000483484.jpg
>693,COCO_train2014_000000305709.jpg
>694,COCO_train2014_000000314646.jpg
>695,COCO_train2014_000000097795.jpg
>696,COCO_train2014_000000051296.jpg
>697,COCO_train2014_000000349603.jpg
>698,COCO_train2014_000000012102.jpg
>699,COCO_train2014_000000145718.jpg
>700,COCO_train2014_000000048848.jpg
>701,COCO_train2014_000000189598.jpg
>702,COCO_train2014_000000390056.jpg
>703,COCO_train2014_000000096820.jpg
>704,COCO_train2014_000000300981.jpg
>705,COCO_train2014_000000369596.jpg
>706,COCO_train2014_000000525580.jpg
>707,COCO_train2014_000000275406.jpg
>708,COCO_train2014_000000013124.jpg
>709,COCO_train2014_000000314893.jpg
>710,COCO_train2014_000000300476.jpg
>711,COCO_train2014_000000022636.jpg
>712,COCO_train2014_000000379980.jpg
>713,COCO_train2014_000000312381.jpg
>714,COCO_train2014_000000326845.jpg
>715,COCO_train2014_000000275316.jpg
>716,COCO_train2014_000000060993.jpg
>

>920,COCO_train2014_000000099402.jpg
>921,COCO_train2014_000000575414.jpg
>922,COCO_train2014_000000466432.jpg
>923,COCO_train2014_000000474930.jpg
>924,COCO_train2014_000000567274.jpg
>925,COCO_train2014_000000396260.jpg
>926,COCO_train2014_000000534195.jpg
>927,COCO_train2014_000000248263.jpg
>928,COCO_train2014_000000492870.jpg
>929,COCO_train2014_000000090145.jpg
>930,COCO_train2014_000000195599.jpg
>931,COCO_train2014_000000314190.jpg
>932,COCO_train2014_000000038411.jpg
>933,COCO_train2014_000000319977.jpg
>934,COCO_train2014_000000581446.jpg
>935,COCO_train2014_000000007781.jpg
>936,COCO_train2014_000000343634.jpg
>937,COCO_train2014_000000486123.jpg
>938,COCO_train2014_000000403305.jpg
>939,COCO_train2014_000000199826.jpg
>940,COCO_train2014_000000455672.jpg
>941,COCO_train2014_000000293342.jpg
>942,COCO_train2014_000000430537.jpg
>943,COCO_train2014_000000206937.jpg
>944,COCO_train2014_000000235734.jpg
>945,COCO_train2014_000000365487.jpg
>946,COCO_train2014_000000032095.jpg
>

...

>17860,COCO_train2014_000000405225.jpg
>17861,COCO_train2014_000000514601.jpg
>17862,COCO_train2014_000000375294.jpg
>17863,COCO_train2014_000000350837.jpg
>17864,COCO_train2014_000000066734.jpg
>17865,COCO_train2014_000000200146.jpg
>17866,COCO_train2014_000000563011.jpg
>17867,COCO_train2014_000000007954.jpg
>17868,COCO_train2014_000000453538.jpg
>17869,COCO_train2014_000000323091.jpg
>17870,COCO_train2014_000000352751.jpg
>17871,COCO_train2014_000000343849.jpg
>17872,COCO_train2014_000000337882.jpg
>17873,COCO_train2014_000000289071.jpg
>17874,COCO_train2014_000000267732.jpg
>17875,COCO_train2014_000000459953.jpg
>17876,COCO_train2014_000000261729.jpg
>17877,COCO_train2014_000000071218.jpg
>17878,COCO_train2014_000000155177.jpg
>17879,COCO_train2014_000000572533.jpg
>17880,COCO_train2014_000000517080.jpg
>17881,COCO_train2014_000000453050.jpg
>17882,COCO_train2014_000000139334.jpg
>17883,COCO_train2014_000000248839.jpg
>17884,COCO_train2014_000000559553.jpg
>17885,COCO_train2014_000

>18080,COCO_train2014_000000022287.jpg
>18081,COCO_train2014_000000483996.jpg
>18082,COCO_train2014_000000152994.jpg
>18083,COCO_train2014_000000246869.jpg
>18084,COCO_train2014_000000079456.jpg
>18085,COCO_train2014_000000075841.jpg
>18086,COCO_train2014_000000064635.jpg
>18087,COCO_train2014_000000010560.jpg
>18088,COCO_train2014_000000115372.jpg
>18089,COCO_train2014_000000399594.jpg
>18090,COCO_train2014_000000026732.jpg
>18091,COCO_train2014_000000240662.jpg
>18092,COCO_train2014_000000493459.jpg
>18093,COCO_train2014_000000449529.jpg
>18094,COCO_train2014_000000173258.jpg
>18095,COCO_train2014_000000117403.jpg
>18096,COCO_train2014_000000253730.jpg
>18097,COCO_train2014_000000290192.jpg
>18098,COCO_train2014_000000554455.jpg
>18099,COCO_train2014_000000557648.jpg
>18100,COCO_train2014_000000512961.jpg
>18101,COCO_train2014_000000025452.jpg
>18102,COCO_train2014_000000535592.jpg
>18103,COCO_train2014_000000251115.jpg
>18104,COCO_train2014_000000472842.jpg
>18105,COCO_train2014_000

>18300,COCO_train2014_000000576685.jpg
>18301,COCO_train2014_000000322411.jpg
>18302,COCO_train2014_000000192137.jpg
>18303,COCO_train2014_000000460565.jpg
>18304,COCO_train2014_000000290485.jpg
>18305,COCO_train2014_000000462613.jpg
>18306,COCO_train2014_000000548974.jpg
>18307,COCO_train2014_000000236740.jpg
>18308,COCO_train2014_000000139285.jpg
>18309,COCO_train2014_000000149566.jpg
>18310,COCO_train2014_000000014125.jpg
>18311,COCO_train2014_000000535218.jpg
>18312,COCO_train2014_000000074415.jpg
>18313,COCO_train2014_000000067118.jpg
>18314,COCO_train2014_000000269680.jpg
>18315,COCO_train2014_000000227502.jpg
>18316,COCO_train2014_000000293674.jpg
>18317,COCO_train2014_000000532269.jpg
>18318,COCO_train2014_000000459325.jpg
>18319,COCO_train2014_000000502197.jpg
>18320,COCO_train2014_000000407135.jpg
>18321,COCO_train2014_000000481889.jpg
>18322,COCO_train2014_000000497455.jpg
>18323,COCO_train2014_000000544660.jpg
>18324,COCO_train2014_000000052069.jpg
>18325,COCO_train2014_000

>18520,COCO_train2014_000000034680.jpg
>18521,COCO_train2014_000000061769.jpg
>18522,COCO_train2014_000000109450.jpg
>18523,COCO_train2014_000000139083.jpg
>18524,COCO_train2014_000000165835.jpg
>18525,COCO_train2014_000000054282.jpg
>18526,COCO_train2014_000000544701.jpg
>18527,COCO_train2014_000000059220.jpg
>18528,COCO_train2014_000000374666.jpg
>18529,COCO_train2014_000000580765.jpg
>18530,COCO_train2014_000000110395.jpg
>18531,COCO_train2014_000000246104.jpg
>18532,COCO_train2014_000000271395.jpg
>18533,COCO_train2014_000000487269.jpg
>18534,COCO_train2014_000000160137.jpg
>18535,COCO_train2014_000000442659.jpg
>18536,COCO_train2014_000000541797.jpg
>18537,COCO_train2014_000000165611.jpg
>18538,COCO_train2014_000000234118.jpg
>18539,COCO_train2014_000000301778.jpg
>18540,COCO_train2014_000000105679.jpg
>18541,COCO_train2014_000000138905.jpg
>18542,COCO_train2014_000000415146.jpg
>18543,COCO_train2014_000000132258.jpg
>18544,COCO_train2014_000000001997.jpg
>18545,COCO_train2014_000

>18740,COCO_train2014_000000239769.jpg
>18741,COCO_train2014_000000079493.jpg
>18742,COCO_train2014_000000458085.jpg
>18743,COCO_train2014_000000230777.jpg
>18744,COCO_train2014_000000537540.jpg
>18745,COCO_train2014_000000222453.jpg
>18746,COCO_train2014_000000074947.jpg
>18747,COCO_train2014_000000319440.jpg
>18748,COCO_train2014_000000112478.jpg
>18749,COCO_train2014_000000493740.jpg
>18750,COCO_train2014_000000276233.jpg
>18751,COCO_train2014_000000284373.jpg
>18752,COCO_train2014_000000407139.jpg
>18753,COCO_train2014_000000294029.jpg
>18754,COCO_train2014_000000017866.jpg
>18755,COCO_train2014_000000072729.jpg
>18756,COCO_train2014_000000309521.jpg
>18757,COCO_train2014_000000402040.jpg
>18758,COCO_train2014_000000145593.jpg
>18759,COCO_train2014_000000402213.jpg
>18760,COCO_train2014_000000392825.jpg
>18761,COCO_train2014_000000406170.jpg
>18762,COCO_train2014_000000036750.jpg
>18763,COCO_train2014_000000371684.jpg
>18764,COCO_train2014_000000175116.jpg
>18765,COCO_train2014_000